
# Análise de Cohort para Portfólio de Análise de Dados

## Introdução

Este projeto tem como objetivo realizar uma análise de cohort detalhada para entender o comportamento de retenção de clientes de uma empresa de varejo online. A análise de cohort é uma ferramenta poderosa que nos permite agrupar usuários com base em uma característica comum (neste caso, o mês da sua primeira compra ou cadastro) e acompanhar o seu comportamento ao longo do tempo. Isso é crucial para identificar tendências de engajamento, retenção e churn, fornecendo insights valiosos para estratégias de negócio.

Este projeto foi desenvolvido como parte de um portfólio de análise de dados, focando em uma abordagem simples, bem estruturada e guiada passo a passo, utilizando apenas ferramentas gratuitas e acessíveis.

## Objetivo

Analisar o comportamento de grupos de usuários (cohortes) ao longo do tempo, baseado na primeira data de compra, para entender retenção e engajamento.

## Ferramentas Utilizadas

*   **Python**: Linguagem de programação principal para manipulação e análise de dados.
*   **Pandas**: Biblioteca para manipulação e análise de dados estruturados.
*   **Matplotlib e Seaborn**: Bibliotecas para visualização de dados, especialmente para a criação do heatmap de retenção.
*   **Google Colab**: Ambiente de desenvolvimento baseado em nuvem, que permite a execução de notebooks Jupyter sem a necessidade de configuração local ou custos.

## Dataset

O dataset utilizado é o **Online Retail Dataset** do UCI Machine Learning Repository. Este conjunto de dados transacionais contém todas as compras realizadas por um varejista online baseado no Reino Unido entre 01/12/2010 e 09/12/2011.




## Fase 1: Preparação do Ambiente e Obtenção dos Dados

Nesta fase, preparamos o ambiente de trabalho e carregamos o dataset. Como estamos utilizando o Google Colab, a instalação de bibliotecas é simplificada. O dataset foi baixado diretamente do repositório UCI.


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar o dataset
# Certifique-se de que o arquivo 'Online Retail.xlsx' está no mesmo diretório do notebook
df = pd.read_excel('Online Retail.xlsx')

# Visualizar as primeiras linhas do dataset para entender sua estrutura
print('Primeiras 5 linhas do dataset:')
print(df.head())

# Visualizar informações básicas do dataset, como tipos de dados e valores não nulos
print('
Informações básicas do dataset:')
print(df.info())

# Visualizar estatísticas descritivas para ter uma ideia da distribuição dos dados numéricos
print('
Estatísticas descritivas do dataset:')
print(df.describe())



## Fase 2: Limpeza e Preparação dos Dados

A qualidade dos dados é fundamental para qualquer análise. Nesta etapa, realizamos a limpeza e preparação do dataset, tratando valores ausentes, convertendo tipos de dados e removendo registros inconsistentes.


In [ ]:

# --- Limpeza e Preparação dos Dados ---

# 1. Tratar valores ausentes
# Remover linhas com CustomerID nulo, pois o ID do cliente é essencial para a análise de cohorte.
# Clientes sem ID não podem ser rastreados ao longo do tempo.
df.dropna(subset=['CustomerID'], inplace=True)

# Preencher valores ausentes em 'Description' com 'Unknown'.
# Embora não seja crítico para a análise de cohorte, é uma boa prática para manter a integridade dos dados.
df['Description'].fillna('Unknown', inplace=True)

# 2. Converter tipos de dados
# Converter CustomerID para int. Ele era float devido à presença de NaNs antes da remoção.
df['CustomerID'] = df['CustomerID'].astype(int)

# A coluna InvoiceDate já está no formato datetime64[ns], conforme verificado com df.info().
# Não é necessária conversão adicional.

# 3. Tratar duplicatas
# Remover linhas duplicadas para garantir que cada transação seja única.
df.drop_duplicates(inplace=True)

# 4. Remover transações com quantidade negativa
# Quantidades negativas geralmente indicam devoluções, que não são relevantes para a análise de retenção de compra.
df = df[df['Quantity'] > 0]

# 5. Remover transações com preço unitário zero ou negativo
# Preços zero ou negativos são dados inconsistentes e podem distorcer a análise.
df = df[df['UnitPrice'] > 0]

# Visualizar informações do dataset após a limpeza para confirmar as alterações
print("
Informações do dataset após a limpeza:")
print(df.info())

# Visualizar as primeiras linhas do dataset após a limpeza
print("
Primeiras 5 linhas do dataset após a limpeza:")
print(df.head())



## Fase 3: Criação das Cohortes e Cálculo de Retenção

Nesta fase, definimos as cohortes e calculamos a matriz de retenção. Uma cohorte é definida pelo mês da primeira compra do cliente. Em seguida, calculamos quantos clientes de cada cohorte retornaram em meses subsequentes.


In [ ]:

# --- Criação das Cohortes e Cálculo de Retenção ---

# Função auxiliar para extrair o mês e ano de uma data no formato 'YYYY-MM'
def get_month(x):
    return x.strftime("%Y-%m")

# Criar uma nova coluna 'InvoiceMonth' com o mês e ano de cada transação
df['InvoiceMonth'] = df['InvoiceDate'].apply(get_month)

# Identificar o mês da primeira compra (mês de cohorte) para cada cliente
# Agrupamos por CustomerID e pegamos o valor mínimo de InvoiceMonth, que será o mês de cohorte.
df['CohortMonth'] = df.groupby('CustomerID')['InvoiceMonth'].transform('min')

# Função auxiliar para calcular a diferença em meses entre duas datas 'YYYY-MM'
def get_month_difference(cohort_month, invoice_month):
    # Extrair ano e mês da string
    y_cohort, m_cohort = int(cohort_month[:4]), int(cohort_month[5:])
    y_invoice, m_invoice = int(invoice_month[:4]), int(invoice_month[5:])
    # Calcular a diferença em meses
    return (y_invoice - y_cohort) * 12 + (m_invoice - m_cohort)

# Criar 'CohortIndex', que representa o número de meses desde a primeira compra para cada transação
df['CohortIndex'] = df.apply(lambda x: get_month_difference(x['CohortMonth'], x['InvoiceMonth']), axis=1)

# Contar o número de usuários únicos por CohortMonth e CohortIndex
# Isso nos dará o número de clientes ativos em cada mês de cada cohorte.
cohort_counts = df.groupby(['CohortMonth', 'CohortIndex'])['CustomerID'].nunique().reset_index()

# Criar a tabela pivô para a retenção
# Onde as linhas são os meses de cohorte, as colunas são os índices de cohorte e os valores são o número de clientes.
cohort_pivot = cohort_counts.pivot_table(index='CohortMonth', columns='CohortIndex', values='CustomerID')

# Calcular a porcentagem de retenção
# O tamanho da cohorte original (mês 0) é usado como base para o cálculo da retenção.
cohort_sizes = cohort_pivot.iloc[:, 0]
retention_matrix = cohort_pivot.divide(cohort_sizes, axis=0)

# Exibir a matriz de retenção (opcional, para ver os valores brutos)
print("
Matriz de Retenção (porcentagem):
")
print(retention_matrix.round(3) * 100)



## Fase 4: Visualização da Análise de Cohort

Para facilitar a interpretação dos resultados, visualizamos a matriz de retenção como um heatmap. O heatmap permite identificar rapidamente padrões de retenção, como a queda inicial e a estabilização ao longo do tempo.


In [ ]:

# --- Visualização da Análise de Cohort ---

plt.figure(figsize=(16, 10))

# Criar o heatmap
# annot=True para exibir os valores nas células
# fmt=".0%" para formatar os valores como porcentagem sem casas decimais
# cmap="YlGnBu" para um esquema de cores que vai do amarelo (baixo) ao azul (alto)
# cbar_kws para personalizar a barra de cores
sns.heatmap(retention_matrix, annot=True, fmt=".0%", cmap="YlGnBu",
            cbar_kws={"label": "Retenção de Cohorte (%)"})

# Adicionar título e rótulos aos eixos
plt.title("Análise de Retenção de Cohorte (por Mês)")
plt.xlabel("Meses desde a Primeira Compra")
plt.ylabel("Mês da Cohorte")

# Rotacionar os rótulos dos eixos para melhor legibilidade
plt.yticks(rotation=0)
plt.xticks(rotation=45)

# Ajustar o layout para evitar sobreposição de elementos
plt.tight_layout()

# Salvar o heatmap como uma imagem PNG
plt.savefig("cohort_retention_heatmap.png")
print("Heatmap de retenção salvo como cohort_retention_heatmap.png")

# Exibir o plot
plt.show()



## Fase 5: Interpretação dos Resultados e Sugestões de Negócio

Após a visualização, é crucial interpretar os padrões observados no heatmap e traduzi-los em insights acionáveis para o negócio. Esta seção detalha as observações e propõe estratégias para melhorar a retenção de clientes.

# Análise de Cohort do Online Retail Dataset

## Introdução

Este projeto tem como objetivo analisar o comportamento de retenção de clientes de uma empresa de varejo online, utilizando a metodologia de análise de cohort. A análise de cohort permite agrupar usuários com base em uma característica comum (neste caso, o mês da primeira compra) e acompanhar o seu comportamento ao longo do tempo. Isso nos ajuda a entender como diferentes grupos de clientes se engajam com o negócio e a identificar tendências de retenção.

## Metodologia

O dataset utilizado é o "Online Retail Dataset" do UCI Machine Learning Repository, que contém dados transacionais de um varejista online do Reino Unido entre 01/12/2010 e 09/12/2011.

As etapas seguidas para a análise foram:

1.  **Preparação e Limpeza dos Dados**: Tratamento de valores ausentes (especialmente `CustomerID`), conversão de tipos de dados, remoção de duplicatas e filtragem de transações inválidas (quantidades e preços negativos/zero).
2.  **Criação das Cohortes**: Cada cliente foi atribuído a uma cohorte com base no mês da sua primeira compra (`CohortMonth`).
3.  **Cálculo do Índice de Cohorte**: Para cada transação, foi calculado o número de meses decorridos desde o `CohortMonth` do cliente (`CohortIndex`).
4.  **Cálculo da Retenção**: A retenção mensal foi calculada como a porcentagem de clientes de uma cohorte que permaneceram ativos (realizaram uma compra) em meses subsequentes, em relação ao tamanho original da cohorte.
5.  **Visualização**: Os resultados foram apresentados em um heatmap, que visualiza a matriz de retenção, facilitando a identificação de padrões.

## Interpretação dos Resultados (Heatmap de Retenção)

O heatmap gerado (`cohort_retention_heatmap.png`) mostra a porcentagem de retenção para cada cohorte ao longo dos meses. As linhas representam o mês da cohorte (mês da primeira compra), e as colunas representam o número de meses desde a primeira compra (índice de cohorte).

**Observações Gerais:**

*   **Queda Inicial Significativa**: A retenção cai drasticamente do mês 0 (mês da primeira compra, sempre 100%) para o mês 1. Por exemplo, a cohorte de Dezembro de 2010 retém apenas 36.6% dos seus clientes no mês seguinte. Isso é um padrão comum em muitos negócios, mas a magnitude da queda sugere que a primeira experiência do cliente ou as estratégias de engajamento pós-compra inicial podem ser melhoradas.
*   **Estabilização Gradual**: Após a queda inicial, a taxa de retenção tende a se estabilizar, embora em níveis relativamente baixos (geralmente entre 15% e 30%). Isso indica que os clientes que permanecem após o primeiro mês têm uma probabilidade maior de continuar ativos por mais tempo.
*   **Variações entre Cohortes**: Algumas cohortes apresentam melhor desempenho de retenção do que outras. Por exemplo, a cohorte de Dezembro de 2010 parece ter uma retenção ligeiramente melhor e mais consistente em comparação com as cohortes de Janeiro ou Fevereiro de 2011 nos primeiros meses. A cohorte de Dezembro de 2010 também mostra um pico de retenção no mês 11 (50.3%), o que pode indicar um comportamento sazonal ou uma campanha específica que reativou esses clientes.
*   **Impacto Sazonal (Dezembro)**: A cohorte de Dezembro de 2010 (mês de festas) demonstra uma retenção inicial mais alta (36.6% no mês 1) em comparação com as cohortes subsequentes (e.g., Janeiro 22.1%, Fevereiro 18.7%). Isso pode ser atribuído a compras de fim de ano, onde os clientes podem estar mais propensos a fazer compras repetidas em um curto período ou a serem presenteados e, posteriormente, se tornarem clientes diretos.
*   **Retenção a Longo Prazo**: A retenção diminui progressivamente ao longo do tempo. Após 6-7 meses, a maioria das cohortes retém menos de 20% dos seus clientes originais. Isso ressalta a importância de estratégias contínuas de engajamento para manter os clientes a longo prazo.

## Sugestões de Negócio

Com base nesta análise de cohort, as seguintes sugestões podem ser consideradas para melhorar a retenção de clientes:

1.  **Foco na Experiência Pós-Primeira Compra**: Dada a alta taxa de churn após o primeiro mês, é crucial otimizar a experiência do cliente imediatamente após a primeira compra. Isso pode incluir:
    *   **Comunicações de Boas-Vindas**: Enviar e-mails ou mensagens personalizadas com dicas de uso do produto, ofertas para a próxima compra ou convites para programas de fidelidade.
    *   **Suporte ao Cliente Proativo**: Oferecer suporte proativo para garantir que os clientes estejam satisfeitos e que quaisquer problemas sejam resolvidos rapidamente.
    *   **Incentivos para Segunda Compra**: Oferecer descontos ou frete grátis para a segunda compra, incentivando a repetição.

2.  **Segmentação e Personalização**: Identificar as características das cohortes com melhor desempenho (como a de Dezembro de 2010) e tentar replicar as condições ou estratégias que levaram a essa retenção. Além disso, segmentar os clientes com base no seu comportamento inicial e personalizar as comunicações e ofertas para cada segmento.

3.  **Programas de Fidelidade e Recompensa**: Implementar ou aprimorar programas de fidelidade que recompensem os clientes por compras repetidas e engajamento contínuo. Isso pode ajudar a manter a retenção em níveis mais altos após os primeiros meses.

4.  **Campanhas de Reengajamento**: Para clientes que estão prestes a se tornar inativos (por exemplo, após 2-3 meses sem compra), lançar campanhas de reengajamento com ofertas especiais, recomendações de produtos personalizadas ou lembretes de itens no carrinho.

5.  **Análise de Causa Raiz para Churn**: Aprofundar a análise para entender por que os clientes estão saindo. Isso pode envolver pesquisas de satisfação, análise de feedback de clientes ou a identificação de padrões em clientes que não retornam (e.g., produtos específicos, canais de aquisição).

6.  **Otimização de Campanhas Sazonais**: Aproveitar o sucesso de cohortes como a de Dezembro de 2010. Entender o que impulsionou a retenção nesse período e aplicar essas lições a outras épocas do ano ou a eventos promocionais.

## Conclusão

A análise de cohort é uma ferramenta poderosa para entender o ciclo de vida do cliente e a eficácia das estratégias de retenção. Embora o dataset mostre uma queda inicial de retenção, há oportunidades claras para melhorias através de intervenções focadas na experiência pós-compra, personalização e programas de fidelidade. Acompanhar essas métricas de retenção de forma contínua é fundamental para o crescimento sustentável do negócio.




